In [49]:
# Cell 1: Imports and Sigmoid
import numpy as np
import pandas as pd

def sigmoid(x):
    return 1 / (1 + np.exp(-x))


In [53]:
# Cell 2: MatrixFactorization Class
class MatrixFactorization:
    def __init__(self, n_users, n_items, n_factors=20, lr=0.01, reg=0.1, epochs=20):
        self.n_users = n_users
        self.n_items = n_items
        self.n_factors = n_factors
        self.lr = lr
        self.reg = reg
        self.epochs = epochs
        self.user_factors = np.random.normal(scale=0.1, size=(n_users, n_factors))
        self.item_factors = np.random.normal(scale=0.1, size=(n_items, n_factors))

    def _sgd_update(self, u, i):
        pred = sigmoid(self.user_factors[u].dot(self.item_factors[i]))
        err = 1 - pred
        grad_u = err * self.item_factors[i] - self.reg * self.user_factors[u]
        grad_i = err * self.user_factors[u] - self.reg * self.item_factors[i]
        self.user_factors[u] += self.lr * grad_u
        self.item_factors[i] += self.lr * grad_i

    def train(self, interactions):
        for epoch in range(self.epochs):
            np.random.shuffle(interactions)
            for u, i in interactions:
                self._sgd_update(u, i)

    def recommend(self, u, k=9, interacted=set()):
        scores = sigmoid(self.item_factors.dot(self.user_factors[u]))
        scores[list(interacted)] = -np.inf
        top_k = np.argpartition(-scores, k)[:k]
        return top_k[np.argsort(-scores[top_k])]

    def update_one(self, u, i, interacted_update):
        self._sgd_update(u, i)
        if u in interacted_update:
            interacted_update[u].add(i)
        else:
            interacted_update[u] = {i}


In [54]:
# Cell 3: ID Mapping Utilities
def map_ids(df, user_col, item_col):
    unique_users = df[user_col].unique()
    unique_items = df[item_col].unique()
    user2idx = {u: idx for idx, u in enumerate(unique_users)}
    item2idx = {i: idx for idx, i in enumerate(unique_items)}
    return user2idx, item2idx


In [ ]:

users_df = pd.read_csv(r'D:\Code\RecSysProj\data\users.csv')
products_df = pd.read_csv(r'D:\Code\RecSysProj\data\products.csv')
interactions_df = pd.read_csv(r'D:\Code\RecSysProj\data\interactions.csv')

In [55]:
# Cell 4: Example Initial Setup
user2idx, item2idx = map_ids(interactions_df, 'user_id', 'product_id')
idx2user = {idx: u for u, idx in user2idx.items()}
idx2item = {idx: i for i, idx in item2idx.items()}

interactions = [(user2idx[r.user_id], item2idx[r.product_id]) for r in interactions_df.itertuples()]
user_interacted = interactions_df.groupby('user_id')['product_id'].apply(set).to_dict()
user_interacted_idx = {user2idx[u]: {item2idx[i] for i in items} for u, items in user_interacted.items()}

mf = MatrixFactorization(n_users=len(user2idx), n_items=len(item2idx),
                         n_factors=50, lr=0.01, reg=0.1, epochs=30)
mf.train(interactions)


In [56]:
# Cell 5: Recommend for a User
u_idx = user2idx[123]
rec_idxs = mf.recommend(u_idx, k=9, interacted=user_interacted_idx.get(u_idx, set()))
recommendations = [idx2item[i] for i in rec_idxs]
print(recommendations)


[449, 256, 52, 241, 337, 156, 184, 366, 139]


In [ ]:
# Cell 6: Incremental Update Example
# Suppose new_interactions_df = pd.read_csv('new_interactions.csv')
new_interactions = [(user2idx[r.user_id], item2idx[r.product_id]) 
                    for r in new_interactions_df.itertuples()]
mf.update(new_interactions, user_interacted_idx)

# Re-run recommendation:
rec_idxs = mf.recommend(u_idx, k=9, interacted=user_interacted_idx.get(u_idx, set()))
recommendations = [idx2item[i] for i in rec_idxs]
print(recommendations)


In [61]:
# Cell 6: Adding Single New Interaction
# Suppose new interaction (user 123 viewed product 456)

new_user_id = 123
new_product_id = 456

# Map to indices
u_idx = user2idx[new_user_id]
i_idx = item2idx[new_product_id]

# Update model immediately
mf.update_one(u_idx, i_idx, user_interacted_idx)

# Now recommend again
rec_idxs = mf.recommend(u_idx, k=9, interacted=user_interacted_idx.get(u_idx, set()))
new_recommendations = [idx2item[i] for i in rec_idxs]
print(new_recommendations)


[449, 256, 52, 337, 156, 241, 184, 139, 366]


In [62]:
#calculate even_new_recommendations with dummy interactions

even_new_product_id= 456

# Map to indices
i_idx = item2idx[even_new_product_id]

# Update model immediately
mf.update_one(u_idx, i_idx, user_interacted_idx)

mf.update_one(u_idx, i_idx, user_interacted_idx)

mf.update_one(u_idx, i_idx, user_interacted_idx)

mf.update_one(u_idx, i_idx, user_interacted_idx)
# Now recommend again
rec_idxs = mf.recommend(u_idx, k=9, interacted=user_interacted_idx.get(u_idx, set()))
even_new_recommendations = [idx2item[i] for i in rec_idxs]
print(even_new_recommendations)


[449, 256, 52, 337, 184, 156, 139, 241, 408]


In [63]:
even_new_recommendations==new_recommendations

False